Compare speakers

In [ ]:
import logging
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))
sys.path.append(os.path.expanduser(".."))

from spinorama.graph import graph_params_default

from generate_common import cache_load

In [ ]:
df = cache_load(filter=None, smoke_test=True)
df.keys()

In [ ]:
def display_compare(df, graph_filter, graph_params=graph_params_default):
    def augment(dfa, name):
        # print(name)
        namearray = [name for i in range(0, len(dfa))]
        dfa["Speaker"] = namearray
        return dfa

    try:
        norm = normalize
        if graph_filter == "CEA2034":
            norm = normalize_cea2034
        source = pd.concat(
            [
                augment(
                    resample(
                        df[speaker][origin][key][graph_filter], 1000
                    ),  # max 1000 Freq points to minimise space
                    "{0} - {1} - {2}".format(speaker, origin, key),
                )
                for speaker in df.keys()
                for origin in df[speaker].keys()
                for key in df[speaker][origin].keys()
                if graph_filter in df[speaker][origin][key]
                and "CEA2034" in df[speaker][origin][key]
            ]
        )

        speaker1 = "KEF LS50 - ASR"
        speaker2 = "KEF LS50 - Princeton"

        if graph_filter == "CEA2034":
            graph = graph_compare_cea2034(source, graph_params, speaker1, speaker2)
        elif graph_filter in ("On Axis", "Estimated In-Room Response"):
            graph = graph_compare_freq_regression(
                source, graph_params, speaker1, speaker2
            )
        else:
            graph = graph_compare_freq(source, graph_params, speaker1, speaker2)
        return graph
    except ValueError as e:
        logging.warning("failed for {0} with {1}".format(graph_filter, e))
        return None

In [ ]:
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["Freq"], empty="none"
)

In [ ]:
def build_selections(df, speaker1, speaker2):
    speakers = df.Speaker.unique()
    input_dropdown1 = alt.binding_select(options=[s for s in speakers])
    selection1 = alt.selection_single(
        fields=["Speaker"],
        bind=input_dropdown1,
        name="Select right ",
        init={"Speaker": speaker1},
    )
    input_dropdown2 = alt.binding_select(options=[s for s in speakers])
    selection2 = alt.selection_single(
        fields=["Speaker"],
        bind=input_dropdown2,
        name="Select left ",
        init={"Speaker": speaker2},
    )
    selectorsMeasurements = alt.selection_multi(fields=["Measurements"], bind="legend")
    scales = alt.selection_interval(bind="scales")
    return selection1, selection2, selectorsMeasurements, scales


def graph_compare_freq(df, graph_params, speaker1, speaker2):
    selection1, selection2, selectorsMeasurements, scales = build_selections(
        df, speaker1, speaker2
    )
    xaxis = alt.X("Freq:Q", scale=alt.Scale(type="log", domain=[20, 20000], nice=False))
    yaxis = alt.Y("dB:Q", scale=alt.Scale(zero=False, domain=[-40, 10]))
    color = alt.Color("Measurements", type="nominal", sort=None)
    line = (
        alt.Chart(df)
        .encode(
            xaxis,
            yaxis,
            color,
            opacity=alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2)),
        )
        .properties(width=600, height=300)
    )

    points = line.mark_circle(size=100).encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=["Measurements", "Freq", "dB"],
    )
    rules = (
        alt.Chart(df)
        .mark_rule(color="gray")
        .encode(x="Freq:Q")
        .transform_filter(nearest)
    )
    graph1 = (
        (points + line.mark_line())
        .add_selection(selection1)
        .transform_filter(selection1)
    )
    graph2 = (
        (points + line.mark_line(strokeDash=[4, 2]))
        .add_selection(selection2)
        .transform_filter(selection2)
    )

    return (
        alt.layer(graph2, graph1, rules)
        .add_selection(selectorsMeasurements)
        .add_selection(scales)
        .add_selection(nearest)
        .interactive()
    )


def graph_compare_cea2034(df, graph_params, speaker1, speaker2):
    selection1, selection2, selectorsMeasurements, scales = build_selections(
        df, speaker1, speaker2
    )

    x_axis = alt.X(
        "Freq:Q", scale=alt.Scale(type="log", domain=[20, 20000], nice=False)
    )
    y_axis = alt.Y("dB:Q", scale=alt.Scale(zero=False, domain=[-40, 10]))
    color = alt.Color("Measurements", type="nominal", sort=None)
    opacity = alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))

    line = (
        alt.Chart(df)
        .transform_filter(
            alt.FieldOneOfPredicate(
                field="Measurements",
                oneOf=[
                    "On Axis",
                    "Listening Window",
                    "Early Reflections",
                    "Sound Power",
                ],
            )
        )
        .encode(x=x_axis, y=y_axis, color=color, opacity=opacity)
    )
    points = line.mark_circle(size=100).encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=["Measurements", "Freq", "dB"],
    )

    di_axis = alt.Y("dB:Q", scale=alt.Scale(zero=False, domain=[0, 30], nice=False))
    di = (
        alt.Chart(df)
        .transform_filter(
            alt.FieldOneOfPredicate(
                field="Measurements", oneOf=["Early Reflections DI", "Sound Power DI"]
            )
        )
        .encode(x=x_axis, y=di_axis, color=color, opacity=opacity)
    )
    points_di = di.mark_circle(size=100).encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=["Measurements", "Freq", "dB"],
    )

    spin_full = (
        alt.layer(points + line.mark_line(), points_di + di.mark_line(clip=True))
        .resolve_scale(y="independent")
        .properties(width=600, height=300)
    )

    spin_dash = (
        alt.layer(
            points + line.mark_line(strokeDash=[4, 2]),
            points_di + di.mark_line(clip=True, strokeDash=[4, 2]),
        )
        .resolve_scale(y="independent")
        .properties(width=600, height=300)
    )

    line1 = spin_full.add_selection(selection1).transform_filter(selection1)
    line2 = spin_dash.add_selection(selection2).transform_filter(selection2)

    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )
    rules = (
        alt.Chart(df)
        .mark_rule(color="gray")
        .encode(x="Freq:Q")
        .transform_filter(nearest)
    )
    layers = (
        alt.layer(line2, line1, rules)
        .add_selection(selectorsMeasurements)
        .add_selection(scales)
        .add_selection(nearest)
        .interactive()
    )
    return layers


def graph_regression_graph(graph, freq_start, freq_end):
    reg = (
        graph.transform_filter(
            "datum.Freq>{0} & datum.Freq<{1}".format(freq_start, freq_end)
        )
        .transform_regression(
            method="log", on="Freq", regression="dB", extent=[20, 20000]
        )
        .encode(alt.X("Freq:Q"), alt.Y("dB:Q"), color=alt.value("red"))
    )
    return reg


def graph_compare_freq_regression(df, graph_params, speaker1, speaker2):
    selection1, selection2, selectorsMeasurements, scales = build_selections(
        df, speaker1, speaker2
    )
    xaxis = alt.X("Freq:Q", scale=alt.Scale(type="log", domain=[20, 20000], nice=False))
    yaxis = alt.Y("dB:Q", scale=alt.Scale(zero=False, domain=[-40, 10]))
    color = alt.Color("Measurements", type="nominal", sort=None)

    line = alt.Chart(df).encode(
        xaxis,
        yaxis,
        color,
        opacity=alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2)),
    )

    points = line.mark_circle(size=100).encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=["Measurements", "Freq", "dB"],
    )

    rules = (
        alt.Chart(df)
        .mark_rule(color="gray")
        .encode(x="Freq:Q")
        .transform_filter(nearest)
    )

    line1 = line.transform_filter(selection1)
    line2 = line.transform_filter(selection2)

    reg1 = graph_regression_graph(line1, 80, 10000).mark_line()
    reg2 = graph_regression_graph(line2, 80, 10000).mark_line(strokeDash=[4, 2])

    graph1 = line1.mark_line().add_selection(selection1)
    graph2 = line2.mark_line(strokeDash=[4, 2]).add_selection(selection2)

    return (
        alt.layer(graph2, reg2, graph1, reg1, rules)
        .add_selection(selectorsMeasurements)
        .add_selection(scales)
        .add_selection(nearest)
        .interactive()
    )

In [ ]:
display_compare(df, "CEA2034")

In [ ]:
display_compare(df, "Vertical Reflections")

In [ ]:
display_compare(df, "Estimated In-Room Response")

In [ ]:
display_compare(df, "Early Reflections")

In [ ]:
display_compare(df, "Horizontal Reflections")

In [ ]:
display_compare(df, "SPL Horizontal")

In [ ]:
display_compare(df, "SPL Vertical")